In [8]:
import requests
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
with open("input.txt", "wb") as f:
   f.write(response.content)

In [9]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [18]:
print(text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [23]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"There are {vocab_size} unique characters. They're as below:")
print(''.join(chars))


There are 65 unique characters. They're as below:

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [31]:
"""
Create mappings, string to integer (stoi) and integer to string (itos)
"""
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
# encode : string -> list of integers
encode = lambda s: [ stoi[c] for c in s ]
# decode : list of integers -> string
decode = lambda l: ''.join([ itos[i] for i in l ])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world
